In [14]:
import argparse
import os
import time
import random
import pickle

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from Dataset_ml import Dataset
from DMF_conv import DMF
from evaluate_deepcf import evaluate_model



from utils_deepcf import (AverageMeter, BatchDataset, get_optimizer,
                   get_train_instances, get_train_matrix, TestDataset)


import math
import heapq  # for retrieval topK
import multiprocessing
import numpy as np
import torch
from utils_deepcf import TestDataset

In [15]:
dataset = Dataset('Data\ml-1m')

In [16]:
train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives

In [68]:

_testRatings = testRatings
_testNegatives = testNegatives
_K = 10
num_thread = 1

In [13]:
hits, ndcgs = [], []
if num_thread > 1:  # Multi-thread
    pool = multiprocessing.Pool(processes = num_thread)
    res = pool.map(eval_one_rating, range(len(_testRatings)))
    pool.close()
    pool.join()
    hits = [r[0] for r in res]
    ndcgs = [r[1] for r in res]
    
# Single thread
for idx in range(len(_testRatings)):
    (hr, ndcg) = eval_one_rating(idx)
    hits.append(hr)
    ndcgs.append(ndcg)      

NameError: name '_model' is not defined

In [10]:
def eval_one_rating(idx):
    rating = _testRatings[idx]
    items = _testNegatives[idx]
    u = rating[0]
    gtItem = rating[1]
    items.append(gtItem)
    
    # Get prediction scores
    map_item_score = {}
    users = np.full(len(items), u, dtype='int32')
   
    # ---
    isCuda = torch.cuda.is_available()
    dst = TestDataset(users, items)
    ldr = torch.utils.data.DataLoader(dst, batch_size=100, shuffle=False)

    _model.eval()
    predictions = [None] * len(dst)
    total = 0
    with torch.no_grad():
        for ui, ii in ldr:
            if isCuda:
                ui, ii = ui.cuda(), ii.cuda()
            bsz = ui.size(0)
            ri = _model(ui, ii).squeeze().cpu().tolist()
            predictions[total:total+bsz] = ri
    # predictions = _model.predict([users, np.array(items)], 
    #                              batch_size=100, verbose=0)
    # ---
    for i in range(len(items)):
        item = items[i]
        map_item_score[item] = predictions[i]
    items.pop()
    
    # Evaluate top rank list
    ranklist = heapq.nlargest(_K, map_item_score, key=map_item_score.get)
    hr = getHitRatio(ranklist, gtItem)
    ndcg = getNDCG(ranklist, gtItem)
    return (hr, ndcg)

In [21]:
idx = 0

In [10]:
len(_testNegatives[0])

99

In [22]:
rating = _testRatings[idx]
items = _testNegatives[idx]
n_items = items
u = rating[0]
gtItem = rating[1]
items.append(gtItem)

In [57]:
ranklist = [1, 2, 3, 4, 5, 6, 7, 2791, 915, 25]

In [32]:
for i in ranklist:
    if i in n_items:
        print(i)
    else:
        print('없음')

없음
없음
없음
없음
없음
없음
없음
없음
없음
없음


In [52]:
n_items

[1064,
 174,
 2791,
 3373,
 269,
 2678,
 1902,
 3641,
 1216,
 915,
 3672,
 2803,
 2344,
 986,
 3217,
 2824,
 2598,
 464,
 2340,
 1952,
 1855,
 1353,
 1547,
 3487,
 3293,
 1541,
 2414,
 2728,
 340,
 1421,
 1963,
 2545,
 972,
 487,
 3463,
 2727,
 1135,
 3135,
 128,
 175,
 2423,
 1974,
 2515,
 3278,
 3079,
 1527,
 2182,
 1018,
 2800,
 1830,
 1539,
 617,
 247,
 3448,
 1699,
 1420,
 2487,
 198,
 811,
 1010,
 1423,
 2840,
 1770,
 881,
 1913,
 1803,
 1734,
 3326,
 1617,
 224,
 3352,
 1869,
 1182,
 1331,
 336,
 2517,
 1721,
 3512,
 3656,
 273,
 1026,
 1991,
 2190,
 998,
 3386,
 3369,
 185,
 2822,
 864,
 2854,
 3067,
 58,
 2551,
 2333,
 2688,
 3703,
 1300,
 1924,
 3118,
 25,
 25,
 25]

In [53]:
2791 in n_items

True

In [56]:
ranklist

[1, 2, 3, 4, 5, 6, 7, 8, 9, 25]

In [60]:
tp = 0
fp = 0
for item in ranklist:
    if item == gtItem:
        tp += 1
    elif item in n_items:
        fp += 1
    else:
        tp = 0
    

In [62]:
tp

1

In [34]:
29 in n_items

False

In [28]:
# Get prediction scores
map_item_score = {}
users = np.full(len(items), u, dtype='int32')

# ---
isCuda = torch.cuda.is_available()
dst = TestDataset(users, items)
ldr = torch.utils.data.DataLoader(dst, batch_size=100, shuffle=False)

_model.eval()
predictions = [None] * len(dst)
total = 0
with torch.no_grad():
    for ui, ii in ldr:
        if isCuda:
            ui, ii = ui.cuda(), ii.cuda()
        bsz = ui.size(0)
        ri = _model(ui, ii).squeeze().cpu().tolist()
        predictions[total:total+bsz] = ri
# predictions = _model.predict([users, np.array(items)], 
#                              batch_size=100, verbose=0)
# ri는 prediction 값으로 0 , 1값이고 그게 prediction에 담김

for i in range(len(items)):
    item = items[i]
    map_item_score[item] = predictions[i]
items.pop()

# map item score에는 item에 해당 아이템, value에는 해당 아이템에 대한 예측 값
# Evaluate top rank list

ranklist = heapq.nlargest(_K, map_item_score, key=map_item_score.get)

# gtItem은 실제 아이템
# ranklist에 들어가 있는 아이템은 
hr = getHitRatio(ranklist, gtItem)
ndcg = getNDCG(ranklist, gtItem)

In [12]:
items

NameError: name 'items' is not defined

In [11]:
def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

In [12]:
def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

In [64]:
rating = _testRatings[0]
items = _testNegatives[0]
u = rating[0]
gtItem = rating[1]

In [70]:
len(items)

102